In [80]:
import pandas as pd
import plotly.express as px
from sklearn.decomposition import PCA
from sklearn.discriminant_analysis import LinearDiscriminantAnalysis
from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestClassifier
import numpy as np
from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import cross_val_score
import matplotlib.pyplot as plt
from sklearn.utils import shuffle 

In [81]:
with open("MoNA-export-RTX5_Fiehnlib.msp", "r") as file:
    current_spectrum=dict()
    spectra=[]
    for line in file:
        if line.startswith("Name"):
            if current_spectrum:
                spectra.append(current_spectrum)
            current_spectrum=dict()
        if line[0].isdigit():
            mass, intensity = map(float, line.split())
            current_spectrum[int(mass)]=intensity
        if line.startswith(("Name", "DB#", "Formula")):
            name, value = line.split(": ")
            current_spectrum[name]=value.strip()
        if line.startswith("Comments"):
            tms=0
            meox=0
            try:
                ind_start = line.index("derivatization type")
                ind_end = line.index('"', ind_start)
                info_string = line[ind_start+20:ind_end]
                tms_meox=[]
                for value in info_string.split("; "):
                    if value.endswith("TMS"):
                        tms = int(value.split()[0])
                    else:
                        meox = int(value.split()[0])
            except:
                pass
            current_spectrum["TMS"]=tms
            current_spectrum["MEOX"]=meox
    spectra.append(current_spectrum)
    
df = pd.DataFrame(spectra).fillna(0)
#Пропуски заполняем нулями
min_mass = min([x for x in df.columns if type(x)==int])
max_mass = max([x for x in df.columns if type(x)==int])
df = df[["Name", "Formula", "DB#", "TMS", "MEOX"]+list(range(min_mass,max_mass+1))]
#Удобный порядок столбцов
df_new = df[df.MEOX==0].drop(columns="MEOX")
df_new.index = range(len(df_new))
#df_new.TMS = np.array(df_new.TMS.apply(lambda x: x if x<=3 else 3))

In [82]:
with open("msp_out.msp", "r") as file:
    current_spectrum=dict()
    spectra=[]
    for line in file:
        if line.startswith("Name"):
            if current_spectrum:
                spectra.append(current_spectrum)
            current_spectrum=dict()
        if line[0].isdigit():
            mass, intensity = map(float, line.split())
            current_spectrum[int(mass)]=intensity/999*100
        if line.startswith(("Name", "n_tms", "Formula","inchikey_without_tms")):
            name, value = line.split(": ")
            current_spectrum[name]=value.strip()       
    spectra.append(current_spectrum)
df_1 = pd.DataFrame(spectra).fillna(0)
df_1[[82,83,84, 496,495]]=0
#df_1.n_tms = df_1.n_tms.astype("int64").apply(lambda x: x if x<=3 else 3)
df_1.n_tms = df_1.n_tms.astype("int64").apply(lambda x: x)
df_1 = df_1[["Name", "Formula", "inchikey_without_tms", "n_tms"]+list(range(min_mass,max_mass+1))]
df_1.rename(columns = {"n_tms":"TMS"}, inplace=True)

In [83]:
df_new = pd.concat([df_1, df_new]).reset_index().drop(columns="index")

In [84]:
df_new.TMS.unique()

array([ 0,  1,  2,  3,  4,  5,  6,  9,  8,  7, 12, 11, 10], dtype=int64)

In [85]:
df_varm = df_new[range(min_mass,max_mass+1)]
df_new = df_new.rename(columns=str)

In [86]:
def AC(dm, row, L):
    summ = 0
    for m in range(min_mass, L - dm + 1):
        summ += row[m] * row[m + dm]
    return summ


x_CENT = []
x_EVEN = []
x_BASE = []
x_SYMX = []
x_AC14 = []
x_logint1 = [[] for _ in range(max_mass-2 - min_mass + 1)]
x_logint2 = [[] for _ in range(max_mass-2 - min_mass + 1)]
#x_logint73 = [[] for _ in range(max_mass-2 - min_mass + 1)]
x = [[] for _ in range(14)]
for index, row in df_varm.iterrows():
    m_max = row[row > 0].index[-1] // 14
    m_min = (row[row > 0].index[0]) // 14
    for q in range(1, 15):
        summ = 0
        for m in range(6, m_max + 1):
            if 14 * m + q <= max_mass:
                summ += row[14 * m + q]
        x[q - 1].append(summ)
    L = row[row > 0].index[-1]
    mIm = 100 * row[row > 0].index * row[row > 0]
    row.sum() * L
    xcent = 0
    for xi in mIm / (row.sum() * L):
        xcent += xi
    x_CENT.append(xcent)
    I2j = 100 * row[row.index % 2 == 0].sum()
    Isum = row.sum()
    x_EVEN.append(I2j / Isum)
    Ib = row[row == row.max()].index[0]
    Isumm = row.sum()
    x_BASE.append(100 * Ib / Isumm)
    S = []
    feature = []
    F = row[row > 0].index[0]
    for m in range(F, L + 1):
        SY = 0
        for j in range(0, min(m - 1, L - m) + 1):
            if min_mass <= m - j <= max_mass and min_mass <= m + j <= max_mass:
                SY += row[m - j] * row[m + j]
        S.append(SY)
    x_SYMX.append(max(S))
    x_AC14.append(100 * AC(14, row, L) / AC(0, row, L))
    for m in range(min_mass, max_mass-1):
        x_logint1[m - min_mass].append(np.log(max(1, row[m]) / max(1, row[m + 1])))
        x_logint2[m - min_mass].append(np.log(max(1, row[m]) / max(1, row[m + 2])))
#    for m in range(min_mass, max_mass-73):
#        x_logint73[m - min_mass].append(np.log(max(1, row[m]) / max(1, row[m + 73])))
for i in range(len(x)):
    x[i] = np.array(x[i])
    x[i] = x[i] / np.max(x[i]) * 100
columns = ["CENT", "EVEN", "BASE"] + [f"MOD14-{i}" for i in range(1, 15)] + [
    "SYMX", "AC14"
] + [f"LG1-{i}"
     for i in range(min_mass, max_mass-1)] + [f"LG2-{i}" for i in range(min_mass, max_mass-1)]
#+ [f"LG73-{i}" for i in range(min_mass, max_mass-73)]
features = [x_CENT, x_EVEN, x_BASE, *x, x_SYMX, x_AC14, *x_logint1, *x_logint2]
#, *x_logint73]
df_new_varm = pd.DataFrame(
    {key: value
     for key, value in zip(columns, features)})
df_scaled = df_new_varm.join(df_new.loc[:, [str(i) for i in range(min_mass,max_mass+1)]])
#scaler = StandardScaler()
#df_scaled = pd.DataFrame({
#    x: y
#    for x, y in zip(columns+[str(i) for i in range(min_mass,max_mass+1)],
#                    scaler.fit_transform(df_new_varm).transpose())
#})
df_scaled

,CENT,EVEN,BASE,MOD14-1,MOD14-2,MOD14-3,MOD14-4,MOD14-5,MOD14-6,MOD14-7,...,491,492,493,494,495,496,497,498,499,500
0,72.992895,52.455919,69.829345,5.075054,1.736466,0.549313,0.933645,0.088365,0.000000,0.000000,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1,72.684100,24.215420,53.801240,4.145818,0.306435,0.224719,0.300100,0.559647,1.905972,0.709567,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2,59.706299,45.982549,26.332485,4.574696,1.455567,7.440699,6.202067,34.815906,43.922067,38.830438,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
3,64.214780,10.050819,64.870130,0.536097,1.046987,36.154806,4.101367,0.500736,0.084710,0.024468,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
4,47.053123,29.330467,5.369318,9.435311,5.439224,48.689139,40.213404,69.160530,37.611182,100.000000,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1610,61.452350,57.850809,16.259155,28.984989,60.980592,40.199750,80.360120,41.914580,8.852182,1.321262,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1611,45.565575,62.810109,12.198933,29.485347,41.573034,20.699126,100.000000,47.569956,62.897078,19.990213,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1612,42.226866,61.644913,24.507627,7.719800,47.752809,18.052434,19.639880,7.835052,19.610336,18.889160,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1613,44.065540,59.091752,13.610102,12.080057,50.791624,18.901373,30.510170,15.611193,26.895384,47.932469,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [87]:
indices = list(range(931))
new_names = list(df_new["inchikey_without_tms"][:684].apply(lambda x: x[:14]))+indices
df_new['new_Name'] = new_names

In [88]:
len(pd.Series(new_names).unique())

1348

In [93]:
pd.Series(df_new.new_Name).astype("str")+" "+df_new.TMS.astype("str")

0       KZNQNBZMBZJQJO 0
1       SFLSHLFXELFNJZ 0
2       QGXBDMJGAMFCBF 0
3       FGSBNBBHOZHUBO 0
4       HVYWMOMLDIMFJA 0
              ...       
1610               926 1
1611               927 2
1612               928 3
1613               929 3
1614               930 2
Length: 1615, dtype: object

In [461]:
#import re
#names = df_new.Name.str.replace(r', \d?TMS derivative', "")
#names = names.str.replace(r', \d?TMS', "")
#names = names.str.replace(r', \d?O-TMS', "")
#names = names.str.replace(r' \d?TMS', "")
#names = names.str.replace(r', \d?O-TMS', "")
#names = names.str.replace(r', N,O,O,-TMS', "")
#names = names.str.replace(r', N,O,O,-tris-TMS', "")
#names = names.str.replace(r', N,O,O-tris-TMS', "")
#names = names.str.replace(r', O,O-bis-TMS', "")
#names = names.str.replace(r', N,O,O-TMS', "")
#names = names.str.replace(r', di-TMS', "")
#names = names.str.replace(r', N1,N.alpha.,O-tris-TMS', "")
#names = names.str.replace(r", O, O', O'' tri-TMS", "")
#names = names.str.replace(r"-TMS", "")

C:\Users\schne\AppData\Local\Temp/ipykernel_2848/2906617246.py:2: FutureWarning: The default value of regex will change from True to False in a future version.
  names = df_new.Name.str.replace(r', \d?TMS derivative', "")
C:\Users\schne\AppData\Local\Temp/ipykernel_2848/2906617246.py:3: FutureWarning: The default value of regex will change from True to False in a future version.
  names = names.str.replace(r', \d?TMS', "")
C:\Users\schne\AppData\Local\Temp/ipykernel_2848/2906617246.py:4: FutureWarning: The default value of regex will change from True to False in a future version.
  names = names.str.replace(r', \d?O-TMS', "")
C:\Users\schne\AppData\Local\Temp/ipykernel_2848/2906617246.py:5: FutureWarning: The default value of regex will change from True to False in a future version.
  names = names.str.replace(r' \d?TMS', "")
C:\Users\schne\AppData\Local\Temp/ipykernel_2848/2906617246.py:6: FutureWarning: The default value of regex will change from True to False in a future version.
  

In [462]:
#df_new.Name = names + " " + [str(x) for x in df_new.TMS]

In [96]:
df_new["TMS"] = df_new.TMS.astype("int64").apply(lambda x: x if x<=3 else 3)

In [97]:
#df_scaled.drop(columns="LG2-435", inplace=True)
df_to_save = df_scaled.join(df_new["TMS"])
df_to_save["Name"] = pd.Series(df_new.new_Name).astype("str")+" "+df_new.TMS.astype("str")
df_to_save.to_csv("fiehnlib_4_classes.csv")

1588

In [104]:
def remove_values_from_list(the_list, val):
    return [value for value in the_list if value != val]
def split(df, test_size=0.25, seed=0):
    np.random.seed(seed)
    train = pd.DataFrame()
    test = pd.DataFrame()
    ind = set(df.index)
    ind_train = []
    names = list(df.Name)
    while len(train)<=len(df)*(1-test_size):
        name = np.random.choice(names)
        names = remove_values_from_list(names, name)
        added = df[df.Name==name]
        ind_train.extend(list(added.index))
        train = pd.concat([train, added])
    test_ind = list(ind-set(ind_train))
    test = df.iloc[test_ind, :]
    X_train, X_test, y_train, y_test = train.loc[:, :"500"], test.loc[:, :"500"], train.TMS, test.TMS
    return X_train, X_test, y_train, y_test
train_test = [
    split(df_to_save,test_size=0.25,seed=i) for i in range(3)
]

In [62]:
len(train_test[0])+len(train_test[2])

8

In [113]:
len(train_test[0][0])/len(df_to_save)

0.7504643962848297